In [ ]:
import cellcharter as cc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import seaborn as sns
import scanpy as sc
import anndata as ad
import squidpy as sq
import os

In [ ]:
extension = 'png'
save_path = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/paper/plots/figure_6_exploration/{extension}/'
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_current_clr_sample_pp.h5ad")

In [ ]:
sq.pl.spatial_scatter(
    adata, 
    color=f'cellcharter_CN', 
    library_key='image_ID', 
    img=None, 
    title=['TS-373_IMC26_B_001.csv'],
    size=4,
    library_id=['TS-373_IMC26_B_001.csv'],
    dpi=100)
sq.pl.spatial_scatter(
    adata, 
    color=f'Phenotype3', 
    library_key='image_ID', 
    img=None, 
    title=['TS-373_IMC26_B_001.csv'],
    size=4,
    library_id=['TS-373_IMC26_B_001.csv'],
    dpi=100)

In [ ]:
# neighborhood_colors = adata.uns['cellcharter_CN_colors']
# phenotype_colors = adata.uns['Phenotype3_colors']
md = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/metadata/metadata.csv')
md = md[md['Cohort'].isin(['B', 'UB'])]
adata = adata[adata.obs['patient_ID'].isin(md['IMC label'])]

In [ ]:
adata.obs['PFS_group'] = adata.obs['patient_ID'].map(md.set_index('IMC label')['Short_long PFS (< or >  2 yrs)'])
adata.obs['PFS_group'] = adata.obs['PFS_group'].astype('category')

# Diff. NBH enrichment between PFS groups

In [ ]:
cc.gr.diff_nhood_enrichment(
    adata,
    cluster_key=f'cellcharter_CN',
    condition_key='PFS_group',
    library_key='image_ID',
)

In [ ]:
cc.pl.diff_nhood_enrichment(
    adata,
    cluster_key=f'cellcharter_CN',
    condition_key='PFS_group',
    condition_groups=['long_PFS', 'short_PFS'],
    figsize=(4,4),
    fontsize=12,
    save = os.path.join(save_path, 'diff_nhood_enrichment.png')
)

# Lets take specific cellcharter nbh enrichment scores per patient

In [ ]:
cc.gr.nhood_enrichment(
    adata,
    cluster_key='cellcharter_CN',
    pvalues=True,
    n_jobs=8,
    n_perms=250
)

In [ ]:
cc.pl.nhood_enrichment(
    adata,
    cluster_key='cellcharter_CN',
    annotate=True,
    figsize=(4,4),
    significance=0.05,
    fontsize=12,
    vmax=0.03208789256832183,
    vmin=-0.03208789256832183
)

In [ ]:
score_categories = ['adaptive_immune*bone_vasculature', 'adaptive_immune*focal_pc_oxphos', 'focal_pc_oxphos*stroma_adipocyte',
                    'pc_myeloid*focal_pc_oxphos', 'bone_vasculature*focal_pc_oxphos', 'hypoxic_immune*focal_pc_oxphos']
def add_patient(patient_id):
    score_dict[patient_id] = {category: [] for category in score_categories}
score_dict = {}

In [ ]:
for patient in md['IMC label'].unique():
    add_patient(patient)
    adata_subset = adata[adata.obs['patient_ID'] == patient].copy()
    cc.gr.nhood_enrichment(
        adata_subset,
        cluster_key='cellcharter_CN',
    )
    for category in score_categories:
        index_nbh1, index_nbh2 = category.split('*')
        score_dict[patient][category] = pd.DataFrame(adata_subset.uns['cellcharter_CN_nhood_enrichment']['enrichment']).loc[index_nbh1, index_nbh2]
    del adata_subset


In [ ]:
df = pd.DataFrame(score_dict).T
df['patient_ID'] = df.index
pfs_mappping = adata.obs[['PFS_group', 'patient_ID']].drop_duplicates()
pfs_mappping.set_index('patient_ID', inplace=True)
df['pfs'] = df['patient_ID'].map(pfs_mappping['PFS_group'])
df.dropna(inplace=True)
df.sort_values('pfs', inplace=True)
df.set_index('patient_ID', inplace=True)
df

In [ ]:
df[df['pfs'] == 'short_PFS']

In [ ]:
df.to_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/downstream/clinical_correlation/nhood_enrichment_patients.csv')